In [1]:
# Student Name : Abdul Bishar
# Cohort       : 5

<h1> Chef Inc:Case Challenge Part II(Individual Assignment 2)</h1>

In an effort to diversify their revenue stream, Apprentice Chef, Inc. has launched Halfway There, a cross-selling promotion where subscribers receive a half bottle of wine from a local California vineyard every Wednesday (halfway through the work week).The executives at Apprentice Chef also believe this endeavor will create a competitive advantage based on its unique product offering of hard to find local wines.Halfway There has been exclusively offered to all of the customers in the dataset you received, and the executives would like to promote this service to a wider audience.They have tasked you with analyzing their data, developing your top insights, and building a machine learning model to predict which customers will subscribe to this service.


Tasks: 

1) Analyze Apprentice Chef's customer user data

2) Develop insights about the "Halfway There promotion"

3) Build a model to predict cross sell success

<h1>Data Preperation</h1>

<h2>Loading packages</h2>

In [2]:
# Import Packages

import pandas as pd                                                   # data science essentials
import numpy as np                                                    # Numeric Python
import matplotlib.pyplot as plt                                       # data viz
import seaborn as sns                                                 # Enhance data viz   


import statsmodels.formula.api as smf                                 # stats model for regression
from sklearn.model_selection import train_test_split,cross_val_score  # Train Tests Split data 
from sklearn.neighbors import KNeighborsClassifier                    # KNN for Classification
from sklearn.preprocessing import StandardScaler                      # Standard scaler
from sklearn.linear_model import LogisticRegression                   # Logistic Regression
from sklearn.metrics import roc_auc_score                             # AUC curve
from sklearn.ensemble import GradientBoostingClassifier               # Gradient Boosting Regressor
from sklearn.model_selection import GridSearchCV                      # GridSearch
from sklearn.metrics import confusion_matrix                          # Confusion Matrix
from sklearn.metrics import make_scorer                               # customizable scorer
 
# libraries for classification trees
from sklearn.tree import DecisionTreeClassifier                       # classification trees
from sklearn.ensemble import GradientBoostingClassifier               # Gradient Boosting Classifier
from sklearn.ensemble import RandomForestClassifier                   # Random Forest for classification
from sklearn.tree import export_graphviz                              # exports graphics
from sklearn.externals.six import StringIO                            # saves objects in memory
from IPython.display import Image                                     # displays on frontend
from sklearn.tree import export_graphviz                              # exports graphics
import pydotplus                                                      # interprets dot objects 

C:\Users\abdul\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


<h2>Data dictionary</h2>

Contains description of features and can be useful as a reference throughout the analysis 

In [3]:

#reading in data dictionary 
chef_description = pd.read_excel('Apprentice_Chef_Data_Dictionary.xlsx')

#displaying the data dictionary
chef_description

,Column,Data Type,Description
0,REVENUE,float,Total revenue generated over the first year of...
1,CROSS_SELL_SUCCESS,integer,Success of promoting Halfway There (1 = SUCCES...
2,NAME,string,Full name of customer (collected upon initial ...
3,EMAIL,string,Email of customer (collected upon initial regi...
4,FIRST_NAME,string,First name of customer (collected upon initial...
5,FAMILY_NAME,string,Last name of customer (collected upon initial ...
6,TOTAL_MEALS_ORDERED,integer,Total count of meals ordered per customer account
7,UNIQUE_MEALS_PURCH,integer,Count of unique meal sets ordered per customer...
8,CONTACTS_W_CUSTOMER_SERVICE,integer,Count of times a customer made contact with cu...
9,PRODUCT_CATEGORIES_VIEWED,integer,Total number of meal categories viewed (online...


<h2>Dataset inital exploration </h2>


68% of the customers in this data set have already taken advantage of the "Halfway there" promotion

Also noteworthy were 47 missing values for FAMILY_NAME

In [4]:
#Reading in dataset and printing first 5 rows
file = 'Apprentice_Chef_Dataset.xlsx'

original = pd.read_excel(file)

#copy of original dataset
chef = original.copy()

#chef.head(n=5)

# Exploring the data and structure
#chef.info()
#chef.describe()

# Exploring the original variables in the dataset
# chef.columns
# chef.isnull().sum()


customers_used_cross_sell = chef['CROSS_SELL_SUCCESS'].sum()
total_rows = chef['CROSS_SELL_SUCCESS'].count()

cross_sell_percentage = (customers_used_cross_sell/total_rows)*100



<h1>Feature Enigneering</h1>

<h2> Missing value treatment </h2>

The  47 missing values for 'FAMILY_NAME' variable represent  <strong> only 2.4% of observations</strong> with missing values. Therefore it is deemed unnecessary for any valuable or meaningful imputation and left as "Uknown". 

Moreover the the FAMILY_NAME was later <strong> found to be insignificant </strong>in determining whether someone will subscribe to Halfway there

In [5]:
#47 missing values identified for FAMILY_NAME 
fill = 'Unknown'
chef['FAMILY_NAME'] = chef['FAMILY_NAME'].fillna(fill)            


<h2> New Feature: Average price per meal </h2>

A new feature "AVERAGE_PRICE_PER_MEAl" is added (including weekly subscription)

My hypothesis is that people who spend higher on average may be more likely to participate in the promotion

In [6]:
# Add Avg. Price per Meal variable
chef['AVG_PRICE_PER_MEAL'] = chef['REVENUE']/chef['TOTAL_MEALS_ORDERED']

#chef['AVG_PRICE_PER_MEAL'].quantile([0.10,
                                      #0.25,
                                      #0.50,
                                      #0.75,
                                      #0.90])
                
#47 missing values identified for FAMILY_NAME 
fill = 'Unknown'
chef['FAMILY_NAME'] = chef['FAMILY_NAME'].fillna(fill)            



<h2> New Features: Email </h2>

This next cell focuses on splitting emails into 3 seperate categories, peronal,professional and junk


In [7]:
# Dummie variables from the email domain.
# Converting dataset to a dataFrame in order to use .iterrows()
chef_email       = pd.DataFrame(chef['EMAIL'])

placeholder_lst  = []

for index, col in chef_email.iterrows():
    split_email  = chef_email.loc[index, 'EMAIL'].split(sep = '@')
    
    placeholder_lst.append(split_email)
    
email_df         = pd.DataFrame(placeholder_lst)
email_df.columns = ['name', 'domain']


# Domain groups
personal_email     = ['@gmail.com', '@yahoo.com','@protonmail.com']
professional_email = ['@mmm.com', '@amex.com','@apple.com',
                      '@boeing.com','@caterpillar.com',
                      '@chevron.com','@cisco.com','@cocacola.com',
                      '@disney.com','@dupont.com','@exxon.com',
                      '@ge.org','@goldmansacs.com','@homedepot.com',
                      '@ibm.com','@intel.com','@jnj.com',
                      '@jpmorgan.com','@mcdonalds.com','@merck.com',
                      '@microsoft.com','@nike.com','@pfizer.com',
                      '@pg.com','@travelers.com','@unitedtech.com',
                      '@unitedhealth.com','@verizon.com','@visa.com',
                      '@walmart.com']
junk_email          = ['@me.com', '@aol.com', '@hotmail.com', '@live.com',
                       '@msn.com','@passport.com']


# For loop categorising the different email domains
placeholder_lst = []

for domain in email_df['domain']:
    
    if '@' + domain in personal_email:
        placeholder_lst.append('personal')
    elif '@' + domain in professional_email:
        placeholder_lst.append('professional')
    else:
        placeholder_lst.append('junk')
        
# make the columns into a series to append it to original dataset        
email_df['DOMAIN_GROUP'] = pd.Series(placeholder_lst)

# Concatenate the email domains as a new column in the chef DataFrame 
chef['DOMAIN'] = email_df['DOMAIN_GROUP']

# Get dummies from the domain variable and drop the original column
one_hot_DOMAIN = pd.get_dummies(chef['DOMAIN'])

# Remove the old and add the 3 new columns
chef           = chef.join([one_hot_DOMAIN])

<h2> New Feature: Valid emails </h2>

Then next cell creates a binary feature that groups personal and professional emails into one group, Valid emails. Valid emails given a value of 1 and anything else i.e. junk emails are given a value of 0

It was found that <strong> 80% of emails are considered valid and 20% are junk </strong>

In [8]:
#variable to gauge intent by seeing if email is Valid

placeholder_lst = []

for row,col in chef.iterrows():
    if chef.loc[row,'DOMAIN'] == 'junk':
        placeholder_lst.append(0)
    else:
        placeholder_lst.append(1)

# Adding the new variable to the original dataset
chef['VALID_EMAIL'] = pd.Series(placeholder_lst)

#chef['VALID_EMAIL'].value_counts()


<h2> New Features: Numbers of Names </h2>

This next feature developed counts the number of names a user has. My hypothesis is that perhaps people with longer titles may have more royalty and have larger spending power which could affect the promotion success

In [9]:
# Adding variable, counting the number of names in NAME column 

def text_split_feature(col, df, sep=' ', new_col_name=None):
    """
Splits values in a string Series (as part of a DataFrame) and sums the number
of resulting items. Automatically appends summed column to original DataFrame.

PARAMETERS
----------
col          : column to split
df           : DataFrame where column is located
sep          : string sequence to split by, default ' '
new_col_name : name of new column after summing split, default
               'number_of_names'
"""
    
    chef[new_col_name] = 0
    
    
    for index, val in chef.iterrows():
        chef.loc[index, new_col_name] = len(chef.loc[index, col].split(sep = ' '))
        
text_split_feature(col = 'NAME', df = chef, new_col_name = 'NUMBER_NAMES' )                                                                                     

<h2> New Feature: Master class attended </h2>

This next feature is a binary feature that determines whether someone has ever attended a master class before. My hypothesis is that people who are engaged enough to come for a master class may be more likely to participate in the promotion

In [10]:
# Making attending a master class into a binary variable (1= attended master class, 0 = never attended masterclass)

#empty list to be appended at the end
placeholder_lst = []

#for loop to loop over master classes attended and apply binary variables

for row,col in chef.iterrows():
    if chef.loc[row,'MASTER_CLASSES_ATTENDED'] >= 1:
        placeholder_lst.append(1)
    else:
        placeholder_lst.append(0)

# Adding the new variable to the original dataset
chef['ATTENDED_MASTER_CLASS'] = pd.Series(placeholder_lst)
#chef = chef.drop(columns = 'MASTER_CLASSES_ATTENDED')

<h2> New Feature: Noblity </h2>

This next feature was developed from the idea from feature 3.4. This feature looks for certain noble titles in the name column and creates a binary feature for whether a user is noble or not. My hypothesis is that nobles may have more money to spend and are better targets for the promotion

Library of titles are available at <a href = "https://en.wikipedia.org/wiki/Imperial,_royal_and_noble_ranks">This Wikipedia page</a>

By doing addiditional research into <a href = "https://awoiaf.westeros.org/index.php/Hizdahr_zo_Loraq"> The city of westoros</a> it was discovered that titles like "mo" and "zo" were refences to "of" and having a reference of "of" usually refers to "daughter of..." or "Son of..." and thus should also be labelled a Noble

In [11]:
# Creating variable that determines whether individual is NOBLE

#empty list to be appended at the end
placeholder_lst = []

#for loop to loop over names and match noble titles

for row,pattern in chef.iterrows():
    if ' of ' in chef.loc[row,'NAME'] or \
    'lord' in chef.loc[row,'NAME'] or \
    'Lord' in chef.loc[row,'NAME'] or \
    ' mo ' in chef.loc[row,'NAME'] or \
    ' zo ' in chef.loc[row,'NAME'] or \
    ' Mo ' in chef.loc[row,'NAME'] or \
    'Knight' in chef.loc[row, 'NAME'] or \
    'knight'in chef.loc[row, 'NAME']:
        placeholder_lst.append(1)
    else:
        placeholder_lst.append(0)
        

#creating column by appending empty list
chef['NOBLE'] = pd.Series(placeholder_lst)

<h2> New Feature: Willing to try new products </h2>

This next feature is a binary feature that classifies someone as "usually" willing to try new products if more than 20% of the time, they order a meal they've never had before. My hypothesis is that these individuals are more experimental and may be more likely to try out the new cross sell promotion

It was found that <strong> 14% of customers fall under this "usually" willing to try new products group</strong>

In [12]:

#Creating a column for % of unique meals purchased
chef['PERCENT_UNIQUE_MEALS']= round(chef['UNIQUE_MEALS_PURCH']/ chef['TOTAL_MEALS_ORDERED']*100, 2)

placeholder_lst = []


for row,col in chef.iterrows():
    if chef.loc[row,'PERCENT_UNIQUE_MEALS'] >= 20:
        placeholder_lst.append(1)
    else:
        placeholder_lst.append(0)

# Adding the new variable to the original dataset
chef['WILLIGNESS_NEW_PRODUCTS'] = pd.Series(placeholder_lst)

#chef['WILLIGNESS_NEW_PRODUCTS'].value_counts()

<h2> New Feature: "Normally" follows recommendations </h2>

This next feature is a binary feature that classifies whether people "normally" follow recommendations by determining if they follow recommendations   <strong>More than half of the time </strong>

It was found that <strong> 35% of users fall into this "Normally" follows recommendation group </strong>

In [13]:
# Follow recomendations appears to be significant. Create a column based on this 

placeholder_lst = []


for row,col in chef.iterrows():
    if chef.loc[row,'FOLLOWED_RECOMMENDATIONS_PCT'] >= 50:
        placeholder_lst.append(1)
    else:
        placeholder_lst.append(0)

# Adding the new variable to the original dataset
chef['NORMALLY_FOLLOWS_RECOMMENDATIONS'] = pd.Series(placeholder_lst)

#chef['NORMALLY_FOLLOWS_RECOMMENDATIONS'].value_counts()


<h2> New Feature: Ordered Beverages Before </h2>

This next feature is a binary feature that classifies whether people on average, order a drink with their food or not. The highest possible "Meal Only" price is 23 dollars, the only way that someones meal price can be higher than 23 dollars is if they order a drink. My hypothesis is that people who also buy drinks are more likely to participate in the promotion

It was found that 27% of users fall into this group

<strong>Assumption: </strong> It is definitely possible that people order an extra drink that is not wine. This feature assumes all extra drinks are wine



In [14]:
# creating ordered beverage column
chef['ORDERED_BEVERAGES'] = 0

# looping to find Miss.
for index, val in chef.iterrows():
    
    if chef.loc[index, 'AVG_PRICE_PER_MEAL'] > 23:
        chef.loc[index, 'ORDERED_BEVERAGES'] = 1
        
    elif chef.loc[index,'AVG_PRICE_PER_MEAL'] <= 23:
        chef.loc[index, 'ORDERED_BEVERAGES'] = 0
    
    else:
        print('error')
        
        
#chef['ORDERED_BEVERAGES'].value_counts()

<h2> Outliers and thresholds </h2>

Next we look at a visual representation of each feature to study the distribution of values and we will use the "eyeball" method to determine high and low outliers for certain variables

Histograms and Boxplots are available to produce in the next two cells

In [15]:

#creating ONLY numerical version of original dataset by dropping charecter features
numerical_chef = chef.drop(columns = ['NAME', 'EMAIL', 'FIRST_NAME', 'FAMILY_NAME','DOMAIN'])

#double checking that all non numerical objects removed
#numerical_chef.info()

#for loop to print historgram of all columns
#for col in numerical_chef:
    
    #fig, ax = plt.subplots(figsize = (10, 8))
    
    #plt.hist(numerical_chef[col], bins = 100)
    #xlabel = print(f'{col}')
    #plt.show()

In [16]:
#for loop to print boxplot of all columns
#for col in numerical_chef:
    
    #fig, ax = plt.subplots(figsize = (8, 6))
    
    #plt.boxplot(x = numerical_chef[col],
                #data = numerical_chef)
    #xlabel = print(f'{col}')
    #plt.show()

<h3>Outlier features </h3>

Next we create an outlier feature for every feature where we decided to mark outliers for 

NB: If a feature has a high and low outliers they will be grouped together 

In [17]:
# Outliers thresholds determined based on the histograms and scatterplots
revenue_hi                    = 6200
total_meals_hi                = 230
unique_meals_hi               = 10
contact_w_customer_service_hi = 12
avg_time_per_site_hi          = 250
cancel_before_noon_hi         = 7
late_deliveries_hi            = 15
early_delivery_lo             = 0
total_photos_viewed_lo        = 0
follow_recommendations_pct_hi = 40
pc_log_hi                     = 7
pc_log_lo                     = 3
mobile_log_hi                 = 3
mobile_log_lo                 = 0
weekly_plan_hi                = 15
avg_prep_video_hi             = 300
total_photos_viewed_lo        = 0
percent_unique_meals_hi       = 30
avg_meal_price_hi             = 120
follow_recommendations_pct_lo = 10


# REVENUE
chef['out_REVENUE']  = 0
condition_hi = chef.loc[0:,'out_REVENUE'][chef['REVENUE'] 
                                                          > revenue_hi]

chef['out_REVENUE'].replace(to_replace = condition_hi,
                                   value      = 1,
                                   inplace    = True)
# TOTAL_MEALS_ORDERED
chef['out_TOTAL_MEALS_ORDERED']  = 0
condition_hi = chef.loc[0:,'out_TOTAL_MEALS_ORDERED'][chef['TOTAL_MEALS_ORDERED'] 
                                                             > total_meals_hi]

chef['out_TOTAL_MEALS_ORDERED'].replace(to_replace = condition_hi,
                                               value      = 1,
                                               inplace    = True)

# UNIQUE_MEALS_PURCH
chef['out_UNIQUE_MEALS_PURCH']  = 0
condition_hi = chef.loc[0:,'out_UNIQUE_MEALS_PURCH'][chef['UNIQUE_MEALS_PURCH'] 
                                                            > unique_meals_hi]

chef['out_UNIQUE_MEALS_PURCH'].replace(to_replace = condition_hi,
                                              value      = 1,
                                              inplace    = True)

# CONTACTS_W_CUSTOMER_SERVICE
chef['out_CONTACTS_W_CUSTOMER_SERVICE']  = 0
condition_hi = chef.loc[0:,'out_CONTACTS_W_CUSTOMER_SERVICE'][chef['CONTACTS_W_CUSTOMER_SERVICE'] 
                                                                     > contact_w_customer_service_hi]

chef['out_CONTACTS_W_CUSTOMER_SERVICE'].replace(to_replace = condition_hi,
                                                       value      = 1,
                                                       inplace    = True)

# AVG_TIME_PER_SITE_VISIT
chef['out_AVG_TIME_PER_SITE_VISIT']  = 0
condition_hi = chef.loc[0:,'out_AVG_TIME_PER_SITE_VISIT'][chef['AVG_TIME_PER_SITE_VISIT'] 
                                                                 > avg_time_per_site_hi]

chef['out_AVG_TIME_PER_SITE_VISIT'].replace(to_replace = condition_hi,
                                                   value      = 1,
                                                   inplace    = True)

# CANCELLATIONS_BEFORE_NOON
chef['out_CANCELLATIONS_BEFORE_NOON']  = 0
condition_hi = chef.loc[0:,'out_CANCELLATIONS_BEFORE_NOON'][chef['CANCELLATIONS_BEFORE_NOON'] 
                                                                   > cancel_before_noon_hi]

chef['out_CANCELLATIONS_BEFORE_NOON'].replace(to_replace = condition_hi,
                                                     value      = 1,
                                                     inplace    = True)


# LATE_DELIVERIES
chef['out_LATE_DELIVERIES']  = 0
condition_hi = chef.loc[0:,'out_LATE_DELIVERIES'][chef['LATE_DELIVERIES'] 
                                                         > late_deliveries_hi]

chef['out_LATE_DELIVERIES'].replace(to_replace = condition_hi,
                                           value      = 1,
                                           inplace    = True)


# EARLY_DELIVERIES
chef['out_EARLY_DELIVERIES']  = 0
condition_lo = chef.loc[0:,'out_EARLY_DELIVERIES'][chef['EARLY_DELIVERIES'] 
                                                         < early_delivery_lo]

chef['out_EARLY_DELIVERIES'].replace(to_replace = condition_lo,
                                           value      = 1,
                                           inplace    = True)

# TOTAL_PHOTOS_VIEWED
chef['out_TOTAL_PHOTOS_VIEWED']  = 0
condition_lo = chef.loc[0:,'out_TOTAL_PHOTOS_VIEWED'][chef['TOTAL_PHOTOS_VIEWED'] 
                                                         < total_photos_viewed_lo]

chef['out_TOTAL_PHOTOS_VIEWED'].replace(to_replace = condition_lo,
                                           value      = 1,
                                           inplace    = True)


# FOLLOW_REC_PCT
chef['out_FOLLOWED_RECOMMENDATIONS_PCT']  = 0
condition_hi = chef.loc[0:,'out_FOLLOWED_RECOMMENDATIONS_PCT'][chef['FOLLOWED_RECOMMENDATIONS_PCT'] 
                                                                     > follow_recommendations_pct_hi  ]
condition_lo = chef.loc[0:,'out_FOLLOWED_RECOMMENDATIONS_PCT'][chef['FOLLOWED_RECOMMENDATIONS_PCT'] 
                                                < follow_recommendations_pct_lo ]


chef['out_FOLLOWED_RECOMMENDATIONS_PCT'].replace(to_replace = condition_hi,
                                                       value      = 1,
                                                       inplace    = True)

chef['out_FOLLOWED_RECOMMENDATIONS_PCT'].replace(to_replace = condition_lo,
                                  value      = 1,
                                  inplace    = True)


# PC_LOGINS
chef['out_PC_LOGINS'] = 0
condition_hi = chef.loc[0:,'out_PC_LOGINS'][chef['PC_LOGINS'] 
                                            > pc_log_hi ]
condition_lo = chef.loc[0:,'out_PC_LOGINS'][chef['PC_LOGINS'] 
                                                < pc_log_lo ]

chef['out_PC_LOGINS'].replace(to_replace = condition_hi,
                              value      = 1,
                              inplace    = True)
chef['out_PC_LOGINS'].replace(to_replace = condition_lo,
                             value      = 1,
                             inplace    = True)


# MOBILE_LOGINS
chef['out_MOBILE_LOGINS'] = 0
condition_hi = chef.loc[0:,'out_MOBILE_LOGINS'][chef['MOBILE_LOGINS'] 
                                                > mobile_log_hi ]
condition_lo = chef.loc[0:,'out_MOBILE_LOGINS'][chef['MOBILE_LOGINS'] 
                                                < mobile_log_lo ]

chef['out_MOBILE_LOGINS'].replace(to_replace = condition_hi,
                                  value      = 1,
                                  inplace    = True)
chef['out_MOBILE_LOGINS'].replace(to_replace = condition_lo,
                                  value      = 1,
                                  inplace    = True)

# WEEKLY_PLAN
chef['out_WEEKLY_PLAN'] = 0
condition_hi = chef.loc[0:,'out_WEEKLY_PLAN'][chef['WEEKLY_PLAN'] 
                                              > weekly_plan_hi]

chef['out_WEEKLY_PLAN'].replace(to_replace = condition_hi,
                                value      = 1,
                                inplace    = True)


# AVG_PREP_VID_TIME
chef['out_AVG_PREP_VID_TIME'] = 0
condition_hi = chef.loc[0:,'out_AVG_PREP_VID_TIME'][chef['AVG_PREP_VID_TIME'] 
                                                    > avg_prep_video_hi]


chef['out_AVG_PREP_VID_TIME'].replace(to_replace = condition_hi,
                                      value      = 1,
                                      inplace    = True)



# TOTAL_PHOTOS_VIEWED
chef['out_TOTAL_PHOTOS_VIEWED']  = 0
condition_lo = chef.loc[0:,'out_TOTAL_PHOTOS_VIEWED'][chef['TOTAL_PHOTOS_VIEWED'] 
                                                         < total_photos_viewed_lo]

chef['out_TOTAL_PHOTOS_VIEWED'].replace(to_replace = condition_lo,
                                           value      = 1,
                                           inplace    = True)


# PERCENT_UNIQUE_MEALS
chef['out_PERCENT_UNIQUE_MEALS'] = 0
condition_hi = chef.loc[0:,'out_PERCENT_UNIQUE_MEALS'][chef['PERCENT_UNIQUE_MEALS'] 
                                                    > percent_unique_meals_hi]


chef['out_PERCENT_UNIQUE_MEALS'].replace(to_replace = condition_hi,
                                      value      = 1,
                                      inplace    = True)

# AVG_PRICE_PER_MEAL
chef['out_AVG_PRICE_PER_MEAL']  = 0
condition_hi = chef.loc[0:,'out_AVG_PRICE_PER_MEAL'][chef['AVG_PRICE_PER_MEAL'] 
                                                          > avg_meal_price_hi]

chef['out_AVG_PRICE_PER_MEAL'].replace(to_replace = condition_hi,
                                              value      = 1,
                                              inplace    = True)


<h1>Correlation and variable selection </h1>


<h2>Heatmap </h2



The heatmap was used a inital measure to get a general feeling of distributions of correlations and to see if anything is immediately noticable

In [18]:
# creating correlation matrix
chef_corr = chef.corr().round(2)

# Heatmap gave a good overview over correlation within the dataset
#fig, ax = plt.subplots(figsize  = (20,20))

#sns.heatmap(chef_corr, cmap = 'coolwarm',
            #square = True, annot = True,
            #linecolor = 'black', linewidths = 0.5)
        
        


<h2>Correlation list </h2
    
First we took a look at all variables correlated with cross sell success. It was found that how often people follow recommendations is the strongest correlated variable(s) with cross sell success. Leading me to my first insight

Because of this i also decided to dive deeper and print out the correlations with "followed_recommendation_pct" to see what factors correlate with how much a person follows recommendations

Finally because the type of email domain seems to be the second biggest correlation with cross sell success. I decided to investigate if there is anything that can potentially correlate with whether someone uses a valid email. (No strong AND logical determination found)



In [19]:
# Looking at correlation with CROSS_SELL_SUCCESS
print(chef_corr['CROSS_SELL_SUCCESS'].sort_values(ascending=False))

# Finding correlation for FOLLOWED_RECOMMENDATIONS_PCT 
#print(chef_corr['FOLLOWED_RECOMMENDATIONS_PCT'].sort_values(ascending=False))

#print(chef_corr['VALID_EMAIL'].sort_values(ascending=False))


CROSS_SELL_SUCCESS                  1.00
NORMALLY_FOLLOWS_RECOMMENDATIONS    0.51
out_FOLLOWED_RECOMMENDATIONS_PCT    0.50
FOLLOWED_RECOMMENDATIONS_PCT        0.46
VALID_EMAIL                         0.28
professional                        0.19
NUMBER_NAMES                        0.16
CANCELLATIONS_BEFORE_NOON           0.16
MOBILE_NUMBER                       0.10
TASTES_AND_PREFERENCES              0.08
REFRIGERATED_LOCKER                 0.07
ATTENDED_MASTER_CLASS               0.05
CONTACTS_W_CUSTOMER_SERVICE         0.04
PC_LOGINS                           0.04
PACKAGE_LOCKER                      0.04
personal                            0.04
MASTER_CLASSES_ATTENDED             0.04
out_CANCELLATIONS_BEFORE_NOON       0.03
AVG_PREP_VID_TIME                   0.03
MEDIAN_MEAL_RATING                  0.03
EARLY_DELIVERIES                    0.02
NOBLE                               0.02
LARGEST_ORDER_SIZE                  0.02
out_AVG_PREP_VID_TIME               0.02
LATE_DELIVERIES 

In [20]:
# For loop to print the numeric variable in the right format for Statmodel
#for col in numerical_chef:
    #print(f"chef['{col}'] +")

<h2>Logistic regression </h2
    
It is here where we added all variables to the model and removed insignificant variables (by p-value) one by one.

For several of the new features created the p value deemed the variable to be insignificant. This is because the features used to develop the new features already incapsulated some of this information. Several test were carried out to determine whether the new features were better predictors or the variables the new features were derived from were better and the following combination was selected

Even though the final variables selected do not give the highest "psuedo r-squared" i believe it was better to carry on with only significant variables

<strong> P-value threshold at 0.05 </strong>

In [21]:

# Statsmodel to build a linear regression with p-values
log_chef_test = smf.logit(formula = """chef['CROSS_SELL_SUCCESS']~
                                        
chef['FOLLOWED_RECOMMENDATIONS_PCT'] +
chef['MOBILE_NUMBER'] +
chef['CANCELLATIONS_BEFORE_NOON'] +
chef['TASTES_AND_PREFERENCES'] +
chef['REFRIGERATED_LOCKER'] +
chef['PC_LOGINS'] +
chef['MOBILE_LOGINS'] +
chef['junk'] +
chef['professional'] +
chef['NUMBER_NAMES'] +
chef['NOBLE'] +
chef['WILLIGNESS_NEW_PRODUCTS'] 
        """, data = chef)


result_fitted_chef_test = log_chef_test.fit()

result_fitted_chef_test.summary()

Optimization terminated successfully.
         Current function value: 0.420638
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                               Logit Regression Results                               
======================================================================================
Dep. Variable:     chef['CROSS_SELL_SUCCESS']   No. Observations:                 1946
Model:                                  Logit   Df Residuals:                     1933
Method:                                   MLE   Df Model:                           12
Date:                        Sun, 15 Mar 2020   Pseudo R-squ.:                  0.3299
Time:                                12:00:37   Log-Likelihood:                -818.56
converged:                               True   LL-Null:                       -1221.6
Covariance Type:                    nonrobust   LLR p-value:                8.231e-165
========================================================================================================
                                           coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
Intercept                               -4.3886      0.682     -6.432      0.000      -5.726      -3.051
chef['FOLLOWED_RECOMMENDATIONS_PCT']     0.0575      0.004     16.269      0.000       0.051       0.064
chef['MOBILE_NUMBER']                    0.7838      0.181      4.331      0.000       0.429       1.138
chef['CANCELLATIONS_BEFORE_NOON']        0.2609      0.045      5.847      0.000       0.173       0.348
chef['TASTES_AND_PREFERENCES']           0.4680      0.134      3.494      0.000       0.205       0.730
chef['REFRIGERATED_LOCKER']              0.5104      0.206      2.482      0.013       0.107       0.913
chef['PC_LOGINS']                        0.2124      0.106      2.006      0.045       0.005       0.420
chef['MOBILE_LOGINS']                   -0.2433      0.116     -2.098      0.036      -0.471      -0.016
chef['junk']                            -1.2905      0.157     -8.218      0.000      -1.598      -0.983
chef['professional']                     0.6692      0.144      4.648      0.000       0.387       0.951
chef['NUMBER_NAMES']                     0.8708      0.110      7.927      0.000       0.655       1.086
chef['NOBLE']                           -1.8227      0.386     -4.718      0.000      -2.580      -1.065
chef['WILLIGNESS_NEW_PRODUCTS']         -0.3583      0.177     -2.019      0.043      -0.706      -0.011
========================================================================================================
"""

<h2>Candidate dictionary </h2
    
 Two candidate dictionaries developed. One containing only the significant variables determine in step 4.4. above. The other dictionary contains all variables

In [22]:
# Variable dictionary for signingicant variables and the full dataset

variable_dict = {
    'logit_sig' : ['MOBILE_NUMBER',
                   'CANCELLATIONS_BEFORE_NOON' ,
                   'TASTES_AND_PREFERENCES' ,
                   'PC_LOGINS',
                   'MOBILE_LOGINS' ,
                   'junk',
                   'professional',
                   'NUMBER_NAMES',
                   'NOBLE',
                   'REFRIGERATED_LOCKER' ,
                   'WILLIGNESS_NEW_PRODUCTS',
                   'FOLLOWED_RECOMMENDATIONS_PCT'],
    
    
   'logit_full' : ['REVENUE','TOTAL_MEALS_ORDERED' ,
                   'UNIQUE_MEALS_PURCH' ,'CONTACTS_W_CUSTOMER_SERVICE',
                   'PRODUCT_CATEGORIES_VIEWED' ,'AVG_TIME_PER_SITE_VISIT',
                   'MOBILE_NUMBER' ,'CANCELLATIONS_BEFORE_NOON' ,'CANCELLATIONS_AFTER_NOON',
                   'TASTES_AND_PREFERENCES' ,'PC_LOGINS' ,
                   'MOBILE_LOGINS','WEEKLY_PLAN' ,'EARLY_DELIVERIES','LATE_DELIVERIES',
                   'PACKAGE_LOCKER','REFRIGERATED_LOCKER',
                   'FOLLOWED_RECOMMENDATIONS_PCT' ,'AVG_PREP_VID_TIME' ,
                   'LARGEST_ORDER_SIZE' ,'MEDIAN_MEAL_RATING' ,'AVG_CLICKS_PER_VISIT',
                   'TOTAL_PHOTOS_VIEWED','AVG_PRICE_PER_MEAL','junk' ,'personal',
                   'professional' ,'VALID_EMAIL',
                   'NUMBER_NAMES' ,'ATTENDED_MASTER_CLASS' ,'NOBLE',
                   'PERCENT_UNIQUE_MEALS' , 'WILLIGNESS_NEW_PRODUCTS' ,
                   'NORMALLY_FOLLOWS_RECOMMENDATIONS','ORDERED_BEVERAGES',
                   'out_REVENUE','out_TOTAL_MEALS_ORDERED','out_UNIQUE_MEALS_PURCH',
                   'out_CONTACTS_W_CUSTOMER_SERVICE' ,'out_AVG_TIME_PER_SITE_VISIT','out_CANCELLATIONS_BEFORE_NOON',
                   'out_LATE_DELIVERIES' ,'out_EARLY_DELIVERIES','out_TOTAL_PHOTOS_VIEWED' ,
                   'out_FOLLOWED_RECOMMENDATIONS_PCT','out_PC_LOGINS' ,'out_MOBILE_LOGINS' ,'out_WEEKLY_PLAN',
                   'out_AVG_PREP_VID_TIME','out_PERCENT_UNIQUE_MEALS', 'out_AVG_PRICE_PER_MEAL']
}

In [23]:
# Splitting the dataset into a train and test set for the statsmodel

#chef_data   = chef.drop(columns = 'CROSS_SELL_SUCCESS')

#chef_target = chef.loc[:,'CROSS_SELL_SUCCESS']

# preparing training and testing sets (Training = 75% , Testing = 25%)
#X_train, X_test, y_train, y_test = train_test_split(chef_data,
                                                   #chef_target,
                                                   #test_size = 0.25,
                                                   #random_state = 222,
                                                   #stratify =chef_target)

# merging training data for statsmodels since it doesn't work the same way as sci-kit learn
#chef_train = pd.concat([X_train, y_train], axis = 1)

<h1> Model development </h1>

Created two training and testing sets based on the significant variables listed in the candidate dictionary

Set 1 - stratifys on 'FOLLOWED_RECOMMENDATIONS_PCT' because it is the most important variable and will help the model if the distribution of variables within is equal

Set 2 - The second set scales the explantory data

<h3> Set 1 </h3>

In [24]:
# Seperating data and target variables
chef_data   =  chef.loc[ : , variable_dict['logit_sig']]
chef_target =  chef.loc[ : , 'CROSS_SELL_SUCCESS']


# preparing training and testing sets (Training = 75% , Testing = 25%)
X_train, X_test, y_train, y_test = train_test_split(
            chef_data,
            chef_target,
            random_state = 222,
            test_size    = 0.25,
            stratify     = chef['FOLLOWED_RECOMMENDATIONS_PCT'])

<h3> Set 2 </h3>

In [25]:
# Seperating data and target variables
chef_data   =  chef.loc[ : , variable_dict['logit_sig']]
chef_target =  chef.loc[ : , 'CROSS_SELL_SUCCESS']


# INSTANTIATING StandardScaler()
scaler = StandardScaler()

# FITTING the independent variable data
scaler.fit(chef_data)


# TRANSFORMING the independent variable data
X_scaled     = scaler.transform(chef_data)


# converting to a DataFrame
X_scaled_df  = pd.DataFrame(X_scaled) 


# Train test split with all the scaled data
X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled = train_test_split(
                    X_scaled_df,
                    chef_target,
                    random_state = 222,
                    test_size = 0.25,
                    stratify = chef_target)

<h3> Model performance </h3>

Creating a list that will be called on later to quickly see which model performed best

In [26]:
# Build a list to keep track of model values
model_performance = [['Model', 'Training Accuracy',
                      'Testing Accuracy', 'AUC Value']]

<h2> Testing on significant variables </h2>

Each model will be tested on both sets of training and testing data outlined inn section 5.0

The models used in this Analysis include

1) Logistic

2) CART

3) Gradient Boost 

4) KNN

<h3> Logistic model </h3>

In [27]:
#INSTANTIATING a logistic regression model
logreg = LogisticRegression(solver = 'liblinear',
                                 C = 1,
                      random_state = 222)


# FITTING the training data
logreg_fit = logreg.fit(X_train, y_train)


# PREDICTING based on the testing set
logreg_pred = logreg_fit.predict(X_test)


# SCORING the results
print('Training ACCURACY:', logreg_fit.score(X_train, y_train).round(4))
print('Testing  ACCURACY:', logreg_fit.score(X_test, y_test).round(4))
print('AUC Score        :', roc_auc_score(y_true  = y_test,
                                          y_score = logreg_pred))

# Adding model results to table
model_performance.append(['Logistic Regression w significant var',
                          logreg_fit.score(X_train, y_train).round(4),
                          logreg_fit.score(X_test, y_test).round(4),
                          roc_auc_score(y_true  = y_test,
                                        y_score = logreg_pred).round(4)])

Training ACCURACY: 0.7807
Testing  ACCURACY: 0.7803
AUC Score        : 0.7504980484631647


In [28]:
# INSTANTIATING a logistic regression model
logreg = LogisticRegression(solver = 'liblinear',
                            C = 1,
                            random_state = 222)


# FITTING the training data
logreg_fit = logreg.fit(X_train_scaled, y_train_scaled)


# PREDICTING based on the testing set
logreg_pred = logreg_fit.predict(X_test_scaled)


# SCORING the results
print('Training ACCURACY:', logreg_fit.score(X_train_scaled, y_train_scaled).round(4))
print('Testing  ACCURACY:', logreg_fit.score(X_test_scaled, y_test_scaled).round(4))
print('AUC Score        :', roc_auc_score(y_true  = y_test_scaled,
                                          y_score = logreg_pred))

# Adding model results to table
model_performance.append(['Logistic Regression scaled significant var',
                          logreg_fit.score(X_train_scaled, y_train_scaled).round(4),
                          logreg_fit.score(X_test_scaled, y_test_scaled).round(4),
                          roc_auc_score(y_true  = y_test_scaled,
                                        y_score = logreg_pred).round(4)])

Training ACCURACY: 0.7985
Testing  ACCURACY: 0.7474
AUC Score        : 0.7057479278023084


<h3> CART model</h3>

<h4> Two user defined functions developed  </h4>

display_tree - To print out tree model

feature_importances - To print out bar graph of the most important features in the CART model


In [29]:
########################################
# display_tree
########################################
def display_tree(tree, feature_df, height = 500, width = 800):
    """
    PARAMETERS
    ----------
    tree       : fitted tree model object
        fitted CART model to visualized
    feature_df : DataFrame
        DataFrame of explanatory features (used to generate labels)
    height     : int, default 500
        height in pixels to which to constrain image in html
    width      : int, default 800
        width in pixels to which to constrain image in html
    """

    # visualizing the tree
    dot_data = StringIO()

    
    # exporting tree to graphviz
    export_graphviz(decision_tree      = tree,
                    out_file           = dot_data,
                    filled             = True,
                    rounded            = True,
                    special_characters = True,
                    feature_names      = feature_df.columns)


    # declaring a graph object
    graph = pydotplus.graph_from_dot_data(dot_data.getvalue())


    # creating image
    img = Image(graph.create_png(),
                height = height,
                width  = width)
    
    return img

########################################
# plot_feature_importances
########################################
def plot_feature_importances(model, train, export = False):
    """
    Plots the importance of features from a CART model.
    
    PARAMETERS
    ----------
    model  : CART model
    train  : explanatory variable training data
    export : whether or not to export as a .png image, default False
    """
    
    # declaring the number
    n_features = X_train.shape[1]
    
    # setting plot window
    fig, ax = plt.subplots(figsize=(12,9))
    
    plt.barh(range(n_features), model.feature_importances_, align='center')
    plt.yticks(pd.np.arange(n_features), train.columns)
    plt.xlabel("Feature importance")
    plt.ylabel("Feature")
    
    if export == True:
        plt.savefig('Tree_Leaf_50_Feature_Importance.png')

<h4> Initial CART model on Set 1 </h4>

It was found that the default hyperparameters were creating too large of a tree

In [30]:
# INSTANTIATING a classification tree object
tree_pruned      = DecisionTreeClassifier()             


# FITTING the training data
tree_pruned_fit  = tree_pruned.fit(X_train, y_train)


# PREDICTING test data set
tree_pred = tree_pruned_fit.predict(X_test)


# SCORING the model
print('Training ACCURACY:', tree_pruned_fit.score(X_train, y_train).round(4))
print('Testing  ACCURACY:', tree_pruned_fit.score(X_test, y_test).round(4))
print('AUC Score        :', roc_auc_score(y_true  = y_test,
                                          y_score = tree_pred).round(4))

# Adding model results to table
model_performance.append(['Decision Tree w significant var',
                          tree_pruned_fit.score(X_train, y_train).round(4),
                          tree_pruned_fit.score(X_test, y_test).round(4),
                          roc_auc_score(y_true  = y_test,
                                          y_score = tree_pred).round(4)])

Training ACCURACY: 0.9596
Testing  ACCURACY: 0.7536
AUC Score        : 0.7377


<h4> Hyperparameter tuning for CART model on Set 1 </h4>


The cell was ran and the hyperparameters produced are given below

Tuned Parameters  :<strong> Tuned Parameters  : {'max_depth': 4, 'min_samples_leaf': 0.11, 'min_samples_split': 0.01}
Tuned CV AUC      : 0.7638 </strong> 

In [31]:
# declaring a hyperparameter space
#max_depth          = pd.np.arange(1, 10, 1)
#min_samples_split = pd.np.arange(0.01, 1, 0.05)
#min_samples_leaf = pd.np.arange(0.01, 0.5, 0.05)

# creating a hyperparameter grid
#params = {'max_depth': max_depth,
          #'min_samples_split': min_samples_split,
          #'min_samples_leaf': min_samples_leaf }



# INSTANTIATING the model object without hyperparameters
#tree = DecisionTreeClassifier()



# GridSearchCV object
#tree_cv = GridSearchCV(tree, 
                       #param_grid= params, 
                       #n_jobs= -1, 
                       #cv = 3,
                       #scoring = make_scorer(roc_auc_score,needs_threshold = False))



# FITTING to the FULL DATASET (due to cross-validation)
#tree_cv = tree_cv.fit(X_test, y_test)



# printing the optimal parameters and best score
#print("Tuned Parameters  :", tree_cv.best_params_)
#print("Tuned CV AUC      :", tree_cv.best_score_.round(4))



<h4> Tuned Tree on set 1 </h4>

The best hyperparameters from the cell above were manually entered and used to produce a tuned tree 

Because this was the <strong> Best CART model </strong> there is an option to print a visual of the tree as well as a bar graph of feature importances (followed_recommendations_pct was by far the most important feature)

In [32]:
# INSTANTIATING a logistic regression model with tuned values
tree_tuned = DecisionTreeClassifier( 
                                    max_depth = 4,
                                    min_samples_leaf = 0.11,
                                    min_samples_split = 0.01,
                                    random_state = 222)


# FITTING the training data
tree_tuned_fit  = tree_tuned.fit(X_train, y_train)

# PREDICTING based on the testing set
tree_tuned_pred = tree_tuned_fit.predict(X_test)



# SCORING the results
print('Training ACCURACY:', tree_tuned.score(X_train, y_train).round(4))
print('Testing  ACCURACY:', tree_tuned.score(X_test, y_test).round(4))
print('AUC Score        :', roc_auc_score(y_true  = y_test,
                                  y_score = tree_tuned_pred).round(4))

# Adding model results to table
model_performance.append(['Tuned Tree w significant var',
                          tree_tuned.score(X_train, y_train).round(4),
                          tree_tuned.score(X_test, y_test).round(4),
                          roc_auc_score(y_true  = y_test,
                                          y_score = tree_pred).round(4)])



# calling display_tree
#display_tree(tree       = tree_tuned,
             #feature_df = X_train)

    
    
# calling plot_feature_importances
#plot_feature_importances(tree_tuned_fit,
                         #train = X_train,
                         #export = False)

Training ACCURACY: 0.8081
Testing  ACCURACY: 0.8111
AUC Score        : 0.7723


<h4> Initial CART model on Set 2 </h4>

*Remember set 2 is the scaled version of the training and testing set

It was found that the default hyperparameters were creating too large of a tree

In [33]:
full_tree = DecisionTreeClassifier()


# FITTING the training data
full_tree_fit = full_tree.fit(X_train_scaled, y_train_scaled)


# PREDICTING on new data
full_tree_pred = full_tree_fit.predict(X_test_scaled)


# SCORING the model
print('Training ACCURACY:', full_tree_fit.score(X_train_scaled, y_train_scaled).round(4))
print('Testing  ACCURACY:', full_tree_fit.score(X_test_scaled, y_test_scaled).round(4))
print('AUC Score        :', roc_auc_score(y_true  = y_test_scaled,
                                          y_score = full_tree_pred).round(4))

# Adding model results to table
model_performance.append(['Decision tree w scaled var',
                          full_tree_fit.score(X_train_scaled, y_train_scaled).round(4),
                          full_tree_fit.score(X_test_scaled, y_test_scaled).round(4),
                          roc_auc_score(y_true  = y_test_scaled,
                                          y_score = full_tree_pred).round(4)])



Training ACCURACY: 0.9575
Testing  ACCURACY: 0.7474
AUC Score        : 0.7142


<h4> Hyperparameter tuning for CART model on Set 2 </h4>


The cell was ran and the hyperparameters produced are given below

Tuned Parameters  :<strong> Tuned Parameters  : {'max_depth': 6, 'min_samples_leaf': 0.01, 'min_samples_split': 0.11}
Tuned CV AUC      : 0.8021 </strong> 

In [34]:
# declaring a hyperparameter space
#max_depth          = pd.np.arange(1, 10, 1)
#min_samples_split = pd.np.arange(0.01, 1, 0.05)
#min_samples_leaf = pd.np.arange(0.01, 0.5, 0.05)

# creating a hyperparameter grid
#params = {'max_depth': max_depth,
          #'min_samples_split': min_samples_split,
          #'min_samples_leaf': min_samples_leaf }



# INSTANTIATING the model object without hyperparameters
#tree = DecisionTreeClassifier()



# GridSearchCV object
#tree_cv = GridSearchCV(tree, 
                       #param_grid= params, 
                       #n_jobs= -1, 
                       #cv = 3,
                       #scoring = make_scorer(roc_auc_score,needs_threshold = False))



# FITTING to the FULL DATASET (due to cross-validation)
#tree_cv = tree_cv.fit(X_test_scaled, y_test_scaled)



# printing the optimal parameters and best score
#print("Tuned Parameters  :", tree_cv.best_params_)
#print("Tuned CV AUC      :", tree_cv.best_score_.round(4))



<h4> Tuned Tree on set 2 </h4>

The best hyperparameters from the cell above are used to produce a tuned tree and a option to visual of the tree is printed at the end

This produced the . It seems tuning the tree on the scaled data does not improve the model

In [35]:
# INSTANTIATING the model object withhyperparameters
tree_tuned =  DecisionTreeClassifier( 
                                    max_depth = 6,
                                    min_samples_leaf = 0.11,
                                    min_samples_split = 0.01,
                                    random_state = 222)


# FITTING the training data
tree_tuned_fit  = tree_tuned.fit(X_train_scaled, y_train_scaled)


# PREDICTING based on the testing set
tree_tuned_pred = tree_tuned_fit.predict(X_test_scaled)



# SCORING the results
print('Training ACCURACY:', tree_tuned.score(X_train_scaled, y_train_scaled).round(4))
print('Testing  ACCURACY:', tree_tuned.score(X_test_scaled, y_test_scaled).round(4))
print('AUC Score        :', roc_auc_score(y_true  = y_test_scaled,
                                  y_score = tree_tuned_pred).round(4))

# Adding model results to table
model_performance.append(['Tuned Tree w scaled var',
                          tree_tuned.score(X_train_scaled, y_train_scaled).round(4),
                          tree_tuned.score(X_test_scaled, y_test_scaled).round(4),
                          roc_auc_score(y_true  = y_test_scaled,
                                          y_score = tree_pred).round(4)])

# displaying the tree
# calling display_tree
#display_tree(tree       = tree_tuned,
             #feature_df = X_train_scaled)



Training ACCURACY: 0.817
Testing  ACCURACY: 0.7844
AUC Score        : 0.738


<h3> Gradient Boosting </h3>

Tne next model developed was a using the gradient boost classifier. Again similarly to all models. The model was ran twice  on Set 1 and Set 2

<h4> Intital Gradient Boost on Set 1  </h4>

After trial and error experimentationm, the hyperparameters used from this step managed to produce <strong> the highest AUC score in this analysis (into the 80's!) </strong>

These hyperparameters managed to produce a higher AUC score than the tuned hyperparameters developed in the coming steps

In [36]:
# INSTANTIATING a classification 
g_boost = GradientBoostingClassifier(loss = 'deviance',
                                     criterion = 'mae',
                                     learning_rate =  0.1,
                                     n_estimators = 95,
                                     max_features = 3,
                                     random_state  = 222)

# FITTING the training data
g_boost_fit = g_boost.fit(X_train, y_train)

# PREDICTING on test data
g_boost_pred = g_boost_fit.predict(X_test)

# SCORING the model
print('Training ACCURACY:', g_boost_fit.score(X_train, y_train).round(4))
print('Testing  ACCURACY:', g_boost_fit.score(X_test, y_test).round(4))
print('AUC Score        :', roc_auc_score(y_true  = y_test,
                                          y_score = g_boost_pred).round(4))

# Adding model results to table
model_performance.append(['GradientBoosting w significant var',
                          g_boost_fit.score(X_train, y_train).round(4),
                          g_boost_fit.score(X_test, y_test).round(4),
                          roc_auc_score(y_true  = y_test,
                                          y_score = g_boost_pred).round(4)])

Training ACCURACY: 0.8328
Testing  ACCURACY: 0.8296
AUC Score        : 0.8181


<h4> Gradient Boost Hyperparameter tuning on Set 1  </h4>

The following results were produced

Tuned Parameters <strong>  : {'learning_rate': 0.1, 'max_depth': 1, 'n_estimators': 200}
Tuned Training AUC: 0.7941 </strong> 

In [37]:
# declaring a hyperparameter space
#learn_space     = pd.np.arange(0.1, 1.6, 0.3)
#estimator_space = pd.np.arange(50, 250, 50)
#depth_space     = pd.np.arange(1, 10)


# creating a hyperparameter grid
#param_grid = {'learning_rate' : learn_space,
              #'max_depth'     : depth_space,
              #'n_estimators'  : estimator_space}


# INSTANTIATING the model object without hyperparameters
#full_gbm_grid = GradientBoostingClassifier(random_state = 802)


# GridSearchCV full
#full_gbm_cv = GridSearchCV(estimator  = full_gbm_grid,
                           #param_grid = param_grid,
                           #cv         = 3,
                           #scoring    = make_scorer(roc_auc_score,
                                        #xneeds_threshold = False))


# FITTING to the FULL DATASET (due to cross-validation)
#full_gbm_cv.fit(X_train, y_train)



# printing the optimal parameters and best score
#print("Tuned Parameters  :", full_gbm_cv.best_params_)
#print("Tuned Training AUC:", full_gbm_cv.best_score_.round(4))

<h4> Using hyperparameters on Set 1 </h4>

The hyperparameters used from the step above were manually fed in and suprisingly lead to a lower AUC score

This leads me to think about the limitations of hyperparameter tuning and whether there are other factors/reasonings for this 

In [38]:
# INSTANTIATING a classification 
g_boost = GradientBoostingClassifier(loss = 'deviance',
                                     criterion = 'mae',
                                     learning_rate =  0.1,
                                     n_estimators = 200,
                                     max_depth = 1,
                                     max_features = 3,
                                     random_state  = 222)





# FITTING the training data
g_boost_fit = g_boost.fit(X_train, y_train)

# PREDICTING on test data
g_boost_pred = g_boost_fit.predict(X_test)

# SCORING the model
print('Training ACCURACY:', g_boost_fit.score(X_train, y_train).round(4))
print('Testing  ACCURACY:', g_boost_fit.score(X_test, y_test).round(4))
print('AUC Score        :', roc_auc_score(y_true  = y_test,
                                          y_score = g_boost_pred).round(4))

# Adding model results to table
model_performance.append(['Tuned GradientBoosting w significant var',
                          g_boost_fit.score(X_train, y_train).round(4),
                          g_boost_fit.score(X_test, y_test).round(4),
                          roc_auc_score(y_true  = y_test,
                                          y_score = g_boost_pred).round(4)])

Training ACCURACY: 0.8026
Testing  ACCURACY: 0.7906
AUC Score        : 0.7414


<h4> Intital Gradient Boost on Set 2  </h4>

This model gave a relatively low AUC score. Hyperparameter tuning was done again but did not seem to raise AUC score significantly and was therefore removed from the analysis

In [39]:
# INSTANTIATING a classification 
g_boost = GradientBoostingClassifier(loss = 'deviance',
                                     criterion = 'mae',
                                     learning_rate =  0.1,
                                     n_estimators = 95,
                                     max_features = 3,
                                     random_state  = 222)

# FITTING the training data
g_boost_fit = g_boost.fit(X_train_scaled, y_train_scaled)

# PREDICTING on test data
g_boost_pred = g_boost_fit.predict(X_test_scaled)

# SCORING the model
print('Training ACCURACY:', g_boost_fit.score(X_train_scaled, y_train_scaled).round(4))
print('Testing  ACCURACY:', g_boost_fit.score(X_test_scaled, y_test_scaled).round(4))
print('AUC Score        :', roc_auc_score(y_true  = y_test_scaled,
                                          y_score = g_boost_pred).round(4))

# Adding model results to table
model_performance.append(['GradientBoosting scaled w scaled var',
                          g_boost_fit.score(X_train_scaled, y_train_scaled).round(4),
                          g_boost_fit.score(X_test_scaled, y_test_scaled).round(4),
                          roc_auc_score(y_true  = y_test_scaled,
                                          y_score = g_boost_pred).round(4)])

Training ACCURACY: 0.8424
Testing  ACCURACY: 0.8172
AUC Score        : 0.7791


<h3> Random Forest </h3>

Tne next model developed was a using the Random Forrest classifier. Again similarly to all models. The model was ran twice  on Set 1 and Set 2

<h4> Intital Random Forrest on Set 1  </h4>

The following model was ran with random hyperparameters selected

In [40]:
# INSTANTIATING a classification tree object
rndfor = RandomForestClassifier(criterion = 'gini',
                                bootstrap = True, 
                                max_depth = 4, 
                                n_estimators = 200,
                                min_samples_leaf = 25, 
                                random_state = 222)

# FITTING the training data
rndfor_fit = rndfor.fit(X_train, y_train)

# PREDICTING on test data
rndfor_pred = rndfor_fit.predict(X_test)

# SCORING the model
print('Training ACCURACY:', rndfor_fit.score(X_train, y_train).round(4))
print('Testing  ACCURACY:', rndfor_fit.score(X_test, y_test).round(4))
print('AUC Score        :', roc_auc_score(y_true  = y_test,
                                          y_score = rndfor_pred).round(4))

model_performance.append(['Random Forrest w significant var',
                          rndfor_fit.score(X_train, y_train).round(4),
                          rndfor_fit.score(X_test, y_test).round(4),
                          roc_auc_score(y_true  = y_test,
                                          y_score = rndfor_pred).round(4)])

Training ACCURACY: 0.8245
Testing  ACCURACY: 0.8111
AUC Score        : 0.7478


<h4> Random Forrest Hyperparameter tuning on Set 1  </h4>

The following results were produced

Tuned Parameters  : {'bootstrap': True, 'criterion': 'entropy', 'min_samples_leaf': 11, 'n_estimators': 100, 'warm_start': True}
Tuned Training AUC: 0.7787

In [41]:
# declaring a hyperparameter space
#estimator_space  = pd.np.arange(100, 1100, 250)
#leaf_space       = pd.np.arange(1, 31, 10)
#criterion_space  = ['gini', 'entropy']
#bootstrap_space  = [True, False]
#warm_start_space = [True, False]


# creating a hyperparameter grid
#param_grid = {'n_estimators'     : estimator_space,
              #'min_samples_leaf' : leaf_space,
              #'criterion'        : criterion_space,
              #'bootstrap'        : bootstrap_space,
              #'warm_start'       : warm_start_space}


# INSTANTIATING the model object without hyperparameters
#full_forest_grid = RandomForestClassifier(random_state = 802)


# GridSearchCV object
#full_forest_cv = GridSearchCV(estimator  = full_forest_grid,
                              #param_grid = param_grid,
                              #cv         = 3,
                              #scoring    = make_scorer(roc_auc_score,
                                           #needs_threshold = False))


# FITTING to the FULL DATASET (due to cross-validation)
#full_forest_cv.fit(X_train, y_train)


# printing the optimal parameters and best score
#print("Tuned Parameters  :", full_forest_cv.best_params_)
#print("Tuned Training AUC:", full_forest_cv.best_score_.round(4))

<h4> Using hyperparameters on Set 1 </h4>

The hyperparameters used from the step above were manually fed in 

In [42]:
# INSTANTIATING a classification tree object
rndfor = RandomForestClassifier(criterion = 'entropy',
                                bootstrap = True,  
                                n_estimators = 100,
                                min_samples_leaf = 11,
                                warm_start = True,
                                random_state = 222)

# FITTING the training data
rndfor_fit = rndfor.fit(X_train, y_train)

# PREDICTING on test data
rndfor_pred = rndfor_fit.predict(X_test)

# SCORING the model
print('Training ACCURACY:', rndfor_fit.score(X_train, y_train).round(4))
print('Testing  ACCURACY:', rndfor_fit.score(X_test, y_test).round(4))
print('AUC Score        :', roc_auc_score(y_true  = y_test,
                                          y_score = rndfor_pred).round(4))

model_performance.append(['Tuned Random Forrest w significant var',
                          rndfor_fit.score(X_train, y_train).round(4),
                          rndfor_fit.score(X_test, y_test).round(4),
                          roc_auc_score(y_true  = y_test,
                                          y_score = rndfor_pred).round(4)])

Training ACCURACY: 0.8355
Testing  ACCURACY: 0.8193
AUC Score        : 0.7761


<h4> Intital Random Forrest on Set 2  </h4>

This model gave a relatively low AUC score. Hyperparameter tuning was done again but did not seem to raise AUC score significantly and was therefore removed from the analysis

In [43]:
# INSTANTIATING a classification tree object
rndfor = RandomForestClassifier(criterion = 'gini',
                                bootstrap = True, 
                                max_depth = 4, 
                                n_estimators = 200,
                                min_samples_leaf = 25, 
                                random_state = 222)

# FITTING the training data
rndfor_fit = rndfor.fit(X_train_scaled, y_train_scaled)

# PREDICTING on test data
rndfor_pred = rndfor_fit.predict(X_test_scaled)

# SCORING the model
print('Training ACCURACY:', rndfor_fit.score(X_train_scaled, y_train_scaled).round(4))
print('Testing  ACCURACY:', rndfor_fit.score(X_test_scaled, y_test_scaled).round(4))
print('AUC Score        :', roc_auc_score(y_true  = y_test_scaled,
                                          y_score = rndfor_pred).round(4))

model_performance.append(['Random Forrest scaled w significant var',
                          rndfor_fit.score(X_train_scaled, y_train_scaled).round(4),
                          rndfor_fit.score(X_test_scaled, y_test_scaled).round(4),
                          roc_auc_score(y_true  = y_test_scaled,
                                          y_score = rndfor_pred).round(4)])

Training ACCURACY: 0.8259
Testing  ACCURACY: 0.7926
AUC Score        : 0.7254


<h3> KNN </h3>

NB: No hyperparameter tuning was developed for this model

<h4> User defined functions developed  </h4>

Optimal Neighbors - Computes training and testing results for KNN as well as the optimal number of neighbors



In [44]:
#User defined function to find the optimal number of neighbors
def optimal_neighbors(X_data,
                      y_data,
                      pct_test=0.25,
                      seed=222,
                      response_type='class',
                      max_neighbors=50):
    """
Exhaustively compute training and testing results for KNN across
[1, max_neighbors]. Outputs the maximum test score and (by default) a
visualization of the results.
PARAMETERS
----------
X_data        : explanatory variable data
y_data        : response variable
pct_test      : test size for training and validation from (0,1), default 0.25
seed          : random seed to be used in algorithm, default 802
response_type : type of neighbors algorithm to use, default 'reg'
    Use 'reg' for regression (KNeighborsRegressor)
    Use 'class' for classification (KNeighborsClassifier)
max_neighbors : maximum number of neighbors in exhaustive search, default 20
"""    
    
    
    
    # train-test split
    X_train, X_test, y_train, y_test = train_test_split(X_data,
                                                        y_data,
                                                        test_size = pct_test,
                                                        random_state = seed)


    # creating lists for training set accuracy and test set accuracy
    training_accuracy = []
    test_accuracy = []
    
    
    # setting neighbor range
    neighbors_settings = range(1, max_neighbors + 1)


    for n_neighbors in neighbors_settings:
        # building the model based on response variable type
        if response_type == 'reg':
            clf = KNeighborsRegressor(n_neighbors = n_neighbors)
            clf.fit(X_train, y_train)
            
        elif response_type == 'class':
            clf = KNeighborsClassifier(n_neighbors = n_neighbors)
            clf.fit(X_train, y_train)            
            
        else:
            print("Error: response_type must be 'reg' or 'class'")
        
        
        # recording the training set accuracy
        training_accuracy.append(clf.score(X_train, y_train))
    
        # recording the generalization accuracy
        test_accuracy.append(clf.score(X_test, y_test))
    
    # returning optimal number of neighbors
    print(f"The optimal number of neighbors is: {test_accuracy.index(max(test_accuracy))+1}")
    return test_accuracy.index(max(test_accuracy))+1

<h4> Intital KNN on Set 1  </h4>

The following model was ran with the optimal number of neighbors (33) 

<strong> This produced the second best model </strong> 

In [45]:
# INSTANTIATING a KNN classification model with optimal neighbors
knn_opt = KNeighborsClassifier(n_neighbors = optimal_neighbors(X_train, 
                                                               y_train))


# FITTING the training data
knn_fit = knn_opt.fit(X_train, y_train)


# PREDICTING based on the testing set
knn_pred = knn_fit.predict(X_test)


# SCORING the results
print('Training ACCURACY:', knn_fit.score(X_train, y_train).round(4))
print('Testing  ACCURACY:', knn_fit.score(X_test, y_test).round(4))
print('AUC Score        :', roc_auc_score(y_true  = y_test,
                                          y_score = knn_pred).round(4))

model_performance.append(['KNeighbor w significant var',
                          knn_fit.score(X_train, y_train).round(4),
                          knn_fit.score(X_test, y_test).round(4),
                          roc_auc_score(y_true  = y_test,
                                          y_score = knn_pred).round(4)])

The optimal number of neighbors is: 33
Training ACCURACY: 0.8232
Testing  ACCURACY: 0.7988
AUC Score        : 0.7942


<h4> Intital KNN on Set 2  </h4>

Running the same model on the scaled variables brough the AUC score lower.

In [46]:
# INSTANTIATING a KNN classification model with optimal neighbors
knn_opt = KNeighborsClassifier(n_neighbors = optimal_neighbors(X_train_scaled, 
                                                               y_train_scaled))


# FITTING the training data
knn_fit = knn_opt.fit(X_train_scaled, y_train_scaled)


# PREDICTING based on the testing set
knn_pred = knn_fit.predict(X_test_scaled)


# SCORING the results
print('Training ACCURACY:', knn_fit.score(X_train_scaled, y_train_scaled).round(4))
print('Testing  ACCURACY:', knn_fit.score(X_test_scaled, y_test_scaled).round(4))
print('AUC Score        :', roc_auc_score(y_true  = y_test_scaled,
                                          y_score = knn_pred).round(4))

model_performance.append(['KNeighbor Scaled w scaled var',
                          knn_fit.score(X_train_scaled, y_train_scaled).round(4),
                          knn_fit.score(X_test_scaled, y_test_scaled).round(4),
                          roc_auc_score(y_true  = y_test_scaled,
                                          y_score = knn_pred).round(4)])

The optimal number of neighbors is: 20
Training ACCURACY: 0.804
Testing  ACCURACY: 0.7659
AUC Score        : 0.716


<h2> Testing on <strong> ALL </strong> variables (i.e. logit_full) </h2>

The following tests were done out of experimentation to see if the significant variables we determined in Section 4.5 were better than using all variables. Therefore the 3 best performing model types so far were ran

Since all previous models ran better with Set 1 i.e. stratified with FOLLOWED_RECOMMENDATIONS_PCT, it was decided to save time and computing power and only run this Set with the logit_full. 

Moreover these models were not hyperparameter tuned since we expect to have lower AUC scores here than with the significant variables

In [47]:
# Adding in all variables to see the ensemble models can do a better job with all
chef_data   =  chef[variable_dict['logit_full']]
chef_target =  chef.loc[:,'CROSS_SELL_SUCCESS']


# this is the exact code we were using before
X_train, X_test, y_train, y_test = train_test_split(
            chef_data,
            chef_target,
            random_state = 222,
            test_size    = 0.25,
            stratify     = chef['FOLLOWED_RECOMMENDATIONS_PCT'])

<h3> Random Forrest </h3>

In [48]:
rndfor = RandomForestClassifier(criterion = 'gini',
                                bootstrap = True, 
                                max_depth = 4, 
                                n_estimators = 200,
                                min_samples_leaf = 25, 
                                random_state = 222)

# FITTING the training data
rndfor_fit = rndfor.fit(X_train, y_train)

# PREDICTING on test data
rndfor_pred = rndfor_fit.predict(X_test)

# SCORING the model
print('Training ACCURACY:', rndfor_fit.score(X_train, y_train).round(4))
print('Testing  ACCURACY:', rndfor_fit.score(X_test, y_test).round(4))
print('AUC Score        :', roc_auc_score(y_true  = y_test,
                                          y_score = rndfor_pred).round(4))

model_performance.append(['Random Forrest w all var',
                          rndfor_fit.score(X_train, y_train).round(4),
                          rndfor_fit.score(X_test, y_test).round(4),
                          roc_auc_score(y_true  = y_test,
                                          y_score = rndfor_pred).round(4)])

Training ACCURACY: 0.8286
Testing  ACCURACY: 0.8172
AUC Score        : 0.7481


<h3> KNN </h3>

<strong> Worst performing model in the whole analysis </strong>

In [49]:
# INSTANTIATING a KNN classification model with optimal neighbors
knn_opt = KNeighborsClassifier(n_neighbors = optimal_neighbors(X_train, 
                                                               y_train))


# FITTING the training data
knn_fit = knn_opt.fit(X_train, y_train)


# PREDICTING based on the testing set
knn_pred = knn_fit.predict(X_test)


# SCORING the results
print('Training ACCURACY:', knn_fit.score(X_train, y_train).round(4))
print('Testing  ACCURACY:', knn_fit.score(X_test, y_test).round(4))
print('AUC Score        :', roc_auc_score(y_true  = y_test,
                                          y_score = knn_pred).round(4))

model_performance.append(['KNeighbor w all var',
                          knn_fit.score(X_train, y_train).round(4),
                          knn_fit.score(X_test, y_test).round(4),
                          roc_auc_score(y_true  = y_test,
                                          y_score = knn_pred).round(4)])

The optimal number of neighbors is: 32
Training ACCURACY: 0.6813
Testing  ACCURACY: 0.7166
AUC Score        : 0.5481


<h3> Gradient Boost </h3>

In [50]:
# INSTANTIATING a classification 
g_boost = GradientBoostingClassifier()

# FITTING the training data
g_boost_fit = g_boost.fit(X_train, y_train)

# PREDICTING on test data
g_boost_pred = g_boost_fit.predict(X_test)

# SCORING the model
print('Training ACCURACY:', g_boost_fit.score(X_train, y_train).round(4))
print('Testing  ACCURACY:', g_boost_fit.score(X_test, y_test).round(4))
print('AUC Score        :', roc_auc_score(y_true  = y_test,
                                          y_score = g_boost_pred).round(4))


model_performance.append(['GradientBoosting w all var',
                          g_boost_fit.score(X_train, y_train).round(4),
                          g_boost_fit.score(X_test, y_test).round(4),
                          roc_auc_score(y_true  = y_test,
                                          y_score = g_boost_pred).round(4)])

Training ACCURACY: 0.9088
Testing  ACCURACY: 0.7988
AUC Score        : 0.7779


<h2> Model Summary </h2>


This is a complete list of models and parameters used.

The best model was <strong> Gradient Boost on significant variables with AUC score = 0.8181 </strong>

In [51]:

#taking model performance list and creating a dataframe
model_performance = pd.DataFrame(model_performance)

model_performance.round(3)

,0,1,2,3
0,Model,Training Accuracy,Testing Accuracy,AUC Value
1,Logistic Regression w significant var,0.7807,0.7803,0.7505
2,Logistic Regression scaled significant var,0.7985,0.7474,0.7057
3,Decision Tree w significant var,0.9596,0.7536,0.7377
4,Tuned Tree w significant var,0.8081,0.8111,0.7377
5,Decision tree w scaled var,0.9575,0.7474,0.7142
6,Tuned Tree w scaled var,0.817,0.7844,0.492
7,GradientBoosting w significant var,0.8328,0.8296,0.8181
8,Tuned GradientBoosting w significant var,0.8026,0.7906,0.7414
9,GradientBoosting scaled w scaled var,0.8424,0.8172,0.7791
